# Experiment 009: High-Precision 70.647306 Submission

Verify and submit the high-precision version of the 70.647306 file.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import os
import json

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
BASE_TREE = Polygon(zip(TX, TY))

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

print("Functions defined")

In [ ]:
# Load the high-precision file
high_prec_file = '/home/nonroot/snapshots/santa-2025/21165876936/code/submission.csv'
df = pd.read_csv(high_prec_file)
print(f"Loaded {len(df)} rows")

# Check precision of first few rows
print("\nPrecision check (first 5 rows):")
for i, row in df.head(5).iterrows():
    x_str = row['x']
    y_str = row['y']
    deg_str = row['deg']
    # Count decimal places
    if 's' in str(x_str):
        x_decimals = len(str(x_str).split('.')[-1]) if '.' in str(x_str) else 0
        y_decimals = len(str(y_str).split('.')[-1]) if '.' in str(y_str) else 0
        deg_decimals = len(str(deg_str).split('.')[-1]) if '.' in str(deg_str) else 0
        print(f"  {row['id']}: x={x_decimals} dec, y={y_decimals} dec, deg={deg_decimals} dec")

In [ ]:
# Calculate total score
def calculate_score(df):
    total_score = 0
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            print(f"Warning: N={n} has {len(n_trees)} trees")
            continue
        all_coords = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            coords = np.array(poly.exterior.coords)
            all_coords.append(coords)
        all_coords = np.vstack(all_coords)
        x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
        y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
        side = max(x_range, y_range)
        total_score += side**2 / n
    return total_score

total_score = calculate_score(df)
print(f"Total score: {total_score:.6f}")

In [ ]:
# Check for overlaps
def check_overlaps(df):
    overlaps_found = []
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        n_trees = df[df['id'].str.startswith(prefix)]
        if len(n_trees) != n:
            continue
        polygons = []
        for _, row in n_trees.iterrows():
            x = parse_value(row['x'])
            y = parse_value(row['y'])
            deg = parse_value(row['deg'])
            poly = create_tree_polygon(x, y, deg)
            polygons.append(poly)
        for i in range(len(polygons)):
            for j in range(i+1, len(polygons)):
                if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                    inter = polygons[i].intersection(polygons[j])
                    if inter.area > 1e-10:
                        overlaps_found.append((n, i, j, inter.area))
    return overlaps_found

print("Checking for overlaps...")
overlaps = check_overlaps(df)
if overlaps:
    print(f"Found {len(overlaps)} overlaps!")
    for n, i, j, area in overlaps[:10]:
        print(f"  N={n}: trees {i} and {j} overlap with area {area:.15f}")
else:
    print("No overlaps found!")

In [ ]:
# Copy to submission
import shutil
os.makedirs('/home/submission', exist_ok=True)
shutil.copy(high_prec_file, '/home/submission/submission.csv')
print(f"Copied to /home/submission/submission.csv")

# Verify the copy has high precision
with open('/home/submission/submission.csv', 'r') as f:
    lines = f.readlines()[:5]
    print("\nVerifying copy precision:")
    for line in lines:
        print(f"  {line.strip()}")

In [ ]:
# Save metrics
os.makedirs('/home/code/experiments/009_high_precision', exist_ok=True)
metrics = {'cv_score': total_score}
with open('/home/code/experiments/009_high_precision/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Saved metrics: {metrics}")